# Install libraries and import them

In [ ]:
!pip install huggingface_hub diffusers transformers torchvision accelerate safetensors datasets kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import kornia.color as kcolor
import matplotlib.pyplot as plt
from safetensors.torch import save_file, load_file
import torchvision.transforms as transforms
from safetensors.torch import save_file
from peft import get_peft_model
from huggingface_hub import login, create_repo, HfApi, hf_hub_download
import os
from diffusers import UniPCMultistepScheduler
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
from torchvision.datasets import ImageFolder
import torchvision
from skimage.color import lab2rgb, rgb2lab

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Load the Dataset

In [ ]:
from huggingface_hub import login
# Insert your tokken here
login("hf_..........")

In [ ]:
dataset = load_dataset("nlphuji/mscoco_2014_5k_test_image_text_retrieval", split="test")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/641 [00:00<?, ?B/s]

(…)oco_2014_5k_test_image_text_retrieval.py:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/510M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/311M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Define some helper function to filter the images

In [ ]:
def is_black_and_white(img):
    img_array = np.array(img)
    if len(img_array.shape) < 3 or img_array.shape[2] != 3:
        return True  # Already grayscale
    return np.all(img_array[:, :, 0] == img_array[:, :, 1]) and np.all(img_array[:, :, 1] == img_array[:, :, 2])

In [ ]:
def mean_saturation(img):
    #Calculate the mean saturation of an image in HSV color space.
    img_hsv = img.convert('HSV')
    np_hsv = np.array(img_hsv)
    return np.mean(np_hsv[:, :, 1]) / 255.0  # Normalize to [0, 1]

We decided to only look at the images with saturation > 0.1

In [ ]:
class PreprocessedValidation(Dataset):
    def __init__(self, dataset, image_size):
        self.dataset = dataset  # Hugging Face dataset
        self.image_size = image_size
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.filtered_data = self._filter_images()

    def _filter_images(self):
        filtered_data = []
        for idx in range(len(self.dataset)):
            img_data = self.dataset[idx]["image"]  # Use correct indexing
            img = img_data.convert("RGB")  # Ensure RGB format

            if not is_black_and_white(img) and mean_saturation(img) >= 0.1:
                filtered_data.append(idx)  # Store index instead of full sample

        return filtered_data

    def __len__(self):
        return len(self.filtered_data)

    def __getitem__(self, idx):
        # Retrieve original dataset index
        dataset_idx = self.filtered_data[idx]
        img_data = self.dataset[dataset_idx]["image"]

        img = img_data.convert("RGB")  # Ensure RGB format

        # Apply transformations
        rgb_img = self.transform(img)

        # Convert to grayscale using Kornia
        gray = kcolor.RgbToGrayscale()
        grayscale_img = gray(rgb_img.unsqueeze(0)).squeeze(0)  # Keep dimensions correct

        return rgb_img, grayscale_img

In [ ]:
# Create the dataset object
image_size = 224
ds = PreprocessedValidation(dataset, image_size)

In [ ]:
rgb_image, gray_img = ds[0]
print(rgb_image.shape)

print(len(ds))

torch.Size([3, 224, 224])
4685


# Load the pretrained model

In [ ]:
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Download the Lora weights

In [ ]:
from peft import PeftModel
unet_lora = PeftModel.from_pretrained(pipe.unet, "MarcoBrigo11/SD2-1_LoRA_Colorization_Imagenette")
pipe.unet = unet_lora

adapter_config.json:   0%|          | 0.00/867 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/24.5M [00:00<?, ?B/s]

In [ ]:
pipe = pipe.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

# Count parameters for each model component in the pipeline
total_params = 0
for name, component in pipe.components.items():
    if hasattr(component, "parameters"):
        num_params = count_parameters(component)
        print(f"{name}: {num_params:,} parameters")
        total_params += num_params

print(f"Total parameters in pipeline: {total_params:,}")

vae: 83,653,863 parameters
text_encoder: 340,387,840 parameters
unet: 872,026,052 parameters
Total parameters in pipeline: 1,296,067,755


# Function used for **inference**

In [ ]:
def inference(model, vae, tokenizer, text_encoder, grayscale_image, steps=50, guidance_scale=7.5, color_scale=1.2, device="cuda"):
    """
    Performs iterative inference for grayscale image colorization.

    Parameters:
    - model: The trained UNet model for predicting color residuals.
    - vae: The Variational Autoencoder for encoding/decoding latent representations.
    - tokenizer: Tokenizer for processing input text prompts.
    - text_encoder: Text encoder to convert captions into embeddings.
    - grayscale_image: Input grayscale image tensor (1, 3, H, W).
    - steps: Number of inference steps (T).
    - guidance_scale: Scale factor for classifier-free guidance.
    - color_scale: Final scaling factor for enhancing color intensity.
    - device: Device for computation ("cuda" or "cpu").

    Returns:
    - colorized_image: The final RGB image.
    """

    model.eval()
    grayscale_image = grayscale_image.repeat(1,3,1,1).to(device)

    # Encode grayscale image into latent space
    z_x_prime = vae.encode(grayscale_image).latent_dist.sample() * vae.config.scaling_factor
    z_t_x = z_x_prime.clone()

    # Prepare text embedding (using empty prompt for default colorization)
    positive_prompt = [""]
    negative_prompt = ["grainy black-and-white photo, photo taken on an old box camera, grayscale photography"]
    tok_pos = tokenizer(positive_prompt, return_tensors="pt", padding="max_length", max_length=tokenizer.model_max_length).input_ids.to(device)
    tok_neg = tokenizer(negative_prompt, return_tensors="pt", padding="max_length", max_length=tokenizer.model_max_length).input_ids.to(device)
    positive_text_embed = text_encoder(tok_pos)[0].to(dtype=torch.float32)
    negative_text_embed = text_encoder(tok_neg)[0].to(dtype=torch.float32)

    for t in reversed(range(steps)):
        t_tensor = torch.full((1,), t/steps, dtype=torch.float32, device=device)

        with torch.no_grad():
            # Run model predictions in mixed precision
            pred_c = model(z_t_x, t_tensor, encoder_hidden_states=positive_text_embed)[0]
            pred_c_uncond = model(z_t_x, t_tensor, encoder_hidden_states=negative_text_embed)[0]

            pred_c = pred_c.to(dtype=torch.float32)
            pred_c_uncond = pred_c_uncond.to(dtype=torch.float32)

            z_x = z_t_x + pred_c + guidance_scale * (pred_c - pred_c_uncond)

            z_t_x = z_t_x + (1 / steps) * (z_x - z_x_prime)

    # Apply final color scale adjustment
    z_x = z_x_prime + color_scale * (z_x - z_x_prime)

    # Decode the final latent to RGB image
    colorized_image = vae.decode((z_x / vae.config.scaling_factor)).sample
    colorized_image = (colorized_image / 2 + 0.5).clamp(0, 1)

    # Convert to numpy format
    colorized_image = colorized_image.detach().cpu().squeeze(0).permute(1, 2, 0).numpy() # Shape: (H, W, 3)

    # Replace the grayscale below by converting to LAB
    lab_image = rgb2lab(colorized_image)
    lab_image[:, :, 0] = grayscale_image[0, 0, :, :].cpu().numpy() * 100  # Ensure L is in [0, 100]
    lab_image = lab2rgb(lab_image)

    return lab_image

In [ ]:
rgb_image, gray_img = ds[2] # ds[0] returns a tuple (rgb_image, gray_img)
sample_img = gray_img.unsqueeze(0)  # Get first image as a grayscale tensor and apply unsqueeze

# Test using only MSE

In [ ]:
for i in range(20):
    rgb_image, gray_img = ds[i]
    sample_img = gray_img.unsqueeze(0)

    # Run inference
    colorized_result = inference(
        pipe.unet,
        pipe.vae,
        pipe.tokenizer,
        pipe.text_encoder,
        grayscale_image=sample_img,
        steps=50,
        guidance_scale=1.6,
        color_scale=1
    )

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Display Grayscale Image
    axes[0].imshow(gray_img.squeeze(0), cmap="gray")  # Remove channel dim
    axes[0].set_title("Grayscale Input")
    axes[0].axis("off")

    # Display Colorized Prediction
    axes[1].imshow(colorized_result)  # First image in batch
    axes[1].set_title("Colorized Prediction")
    axes[1].axis("off")

    # Display Original RGB Image
    axes[2].imshow(rgb_image.permute(1, 2, 0).clamp(0, 1))  # Ensure HWC format
    axes[2].set_title("Original RGB")
    axes[2].axis("off")

    plt.suptitle(f"Comparison {i+1}")
    plt.show()

# Test using LPIPS as loss with MSE

In [ ]:
for i in range(20):
    rgb_image, gray_img = ds[i]
    sample_img = gray_img.unsqueeze(0)

    # Run inference
    colorized_result = inference(
        pipe.unet,
        pipe.vae,
        pipe.tokenizer,
        pipe.text_encoder,
        grayscale_image=sample_img,
        steps=50,
        guidance_scale=1.6,
        color_scale=1
    )

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Display Grayscale Image
    axes[0].imshow(gray_img.squeeze(0), cmap="gray")  # Remove channel dim
    axes[0].set_title("Grayscale Input")
    axes[0].axis("off")

    # Display Colorized Prediction
    axes[1].imshow(colorized_result)  # First image in batch
    axes[1].set_title("Colorized Prediction")
    axes[1].axis("off")

    # Display Original RGB Image
    axes[2].imshow(rgb_image.permute(1, 2, 0).clamp(0, 1))  # Ensure HWC format
    axes[2].set_title("Original RGB")
    axes[2].axis("off")

    plt.suptitle(f"Comparison {i+1}")
    plt.show()

# Test using Saturation loss with MSE

In [ ]:
for i in range(20):
    rgb_image, gray_img = ds[i]
    sample_img = gray_img.unsqueeze(0)

    # Run inference
    colorized_result = inference(
        pipe.unet,
        pipe.vae,
        pipe.tokenizer,
        pipe.text_encoder,
        grayscale_image=sample_img,
        steps=50,
        guidance_scale=1.6,
        color_scale=1
    )

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Display Grayscale Image
    axes[0].imshow(gray_img.squeeze(0), cmap="gray")  # Remove channel dim
    axes[0].set_title("Grayscale Input")
    axes[0].axis("off")

    # Display Colorized Prediction
    axes[1].imshow(colorized_result)  # First image in batch
    axes[1].set_title("Colorized Prediction")
    axes[1].axis("off")

    # Display Original RGB Image
    axes[2].imshow(rgb_image.permute(1, 2, 0).clamp(0, 1))  # Ensure HWC format
    axes[2].set_title("Original RGB")
    axes[2].axis("off")

    plt.suptitle(f"Comparison {i+1}")
    plt.show()